### Initial Setup
Make sure you have the following packages installed:
- pandas
- jupysql
- duckdb
- duckdb-engine

`pip install jupysql duckdb duckdb-engine`

In [1]:
# pip install jupysql duckdb duckdb-engine

# TODO: Build a combined database

In [2]:
import duckdb
import json
import pandas as pd
from datetime import datetime, timedelta

# use an in memory database
conn = duckdb.connect(':memory:')

In [3]:
trailing_days = 365
get_start_of_month = True # True if we want to start on the first day of the month

In [4]:
def extract_json_field(json_string, field_path):
    try:
        # Check if the string is already a dictionary
        if isinstance(json_string, dict):
            data = json_string
        else:
            # Replace single quotes with double quotes, 'None' with 'null', and parse
            json_string = json_string.replace("'", '"').replace('None', 'null')
            data = json.loads(json_string)
        
        # Navigate the JSON structure
        for key in field_path.split('.'):
            if key.isdigit():
                key = int(key)
            data = data[key]
        
        return data
    except Exception as e:
        # print(f"Error extracting field {field_path} from {json_string}: {e}")
        return None

def extract_name_by_category(badges_json, target_category):
    if pd.isna(badges_json):
        return None
    
    try:
        badges = json.loads(badges_json.replace("'", '"').replace('None', 'null'))
        for badge in badges:
            if badge.get('category') == target_category:
                return badge.get('name')
        return None  # Return None if no matching category is found
    except Exception as e:
        print(f"Error extracting name for category '{target_category}' from {badges_json}: {e}")
        return None

def extract_jsons(int_df, json_extractions):
    for new_column, (json_column, field_path) in json_extractions.items():
        int_df[new_column] = int_df[json_column].apply(lambda x: extract_json_field(x, field_path))
    return int_df


In [5]:
# Extract json labels

# download files
df_gtp_md = pd.read_csv('downloads/growthepie_metadata.csv')
df_l2b_md = pd.read_csv('downloads/l2beat_summary.csv')

# identify patterns
gtp_json_extractions = {
    'stack_label': ('stack', 'label'),
}

# Execute GTP
df_gtp_md = extract_jsons(df_gtp_md,gtp_json_extractions)

# Ecevute L2B
df_l2b_md['stack_name'] = df_l2b_md['badges'].apply(lambda x: extract_name_by_category(x, 'Stack'))
df_l2b_md['da_layer'] = df_l2b_md['badges'].apply(lambda x: extract_name_by_category(x, 'DA'))
df_l2b_md['vm'] = df_l2b_md['badges'].apply(lambda x: extract_name_by_category(x, 'VM'))
df_l2b_md['infra'] = df_l2b_md['badges'].apply(lambda x: extract_name_by_category(x, 'Infra'))
df_l2b_md['raas'] = df_l2b_md['badges'].apply(lambda x: extract_name_by_category(x, 'RaaS'))

In [6]:
# Create tables from CSV files
conn.execute("CREATE TABLE defillama_chains AS SELECT * FROM 'downloads/defillama_chains.csv'")
conn.execute("CREATE TABLE defillama_tvl AS SELECT * FROM 'downloads/defillama_tvl.csv'")

conn.execute("CREATE TABLE growthepie_fundamentals AS SELECT * FROM 'downloads/growthepie_fundamentals.csv'")
conn.execute("CREATE TABLE growthepie_metadata AS SELECT * FROM df_gtp_md")

conn.execute("CREATE TABLE l2beat_activity AS SELECT * FROM 'downloads/l2beat_activity.csv'")
conn.execute("CREATE TABLE l2beat_assets_onchain AS SELECT * FROM 'downloads/l2beat_assets_onchain.csv'")
conn.execute("CREATE TABLE l2beat_summary AS SELECT * FROM df_l2b_md")

### Intermediate Data x Metadata Views
Join the data tables with the metadata tables for each data source, to then be used for joining

In [7]:
#Build Growthepie Model
conn.execute("""
CREATE OR REPLACE VIEW growthepie AS
SELECT
    cast(f.date as timestamp) AS date
    , f.* EXCLUDE (date)
    , m.* EXCLUDE (origin_key)
    , MIN(cast(f.date as timestamp)) OVER (PARTITION BY f.origin_key) AS gtp_min_date
FROM growthepie_fundamentals f
INNER JOIN growthepie_metadata m
    ON f.origin_key = m.origin_key
WHERE f.date < CURRENT_DATE()
""")

#Build L2Beat Model
conn.execute("""
CREATE OR REPLACE VIEW l2beat AS
SELECT
    COALESCE(s.slug,t.slug, aoc.slug) AS slug
    , cast(COALESCE(t.timestamp, aoc.timestamp) as timestamp) AS date
    , s.* EXCLUDE (slug)
    , t.* EXCLUDE (slug, timestamp)
    , aoc.* EXCLUDE (slug, timestamp)
    , MIN(cast(COALESCE(t.timestamp, aoc.timestamp) as timestamp)) OVER (PARTITION BY COALESCE(s.slug,t.slug, aoc.slug)) AS l2b_min_date
FROM l2beat_summary s
FULL OUTER JOIN l2beat_activity t
    ON s.slug = t.slug
FULL OUTER JOIN l2beat_assets_onchain aoc
    ON s.slug = aoc.slug
    AND t.timestamp = aoc.timestamp
WHERE COALESCE(t.timestamp, aoc.timestamp) < CURRENT_DATE()
""")

#Build Defillama Model
conn.execute("""
CREATE OR REPLACE VIEW defillama AS
SELECT
      cast(tvl.date AS timestamp) as date
    , ch.*
    , tvl.* EXCLUDE (name, chain_id, date)
    , MIN(cast(tvl.date as timestamp)) OVER (PARTITION BY tvl.name) AS dfl_min_date
FROM defillama_chains ch
INNER JOIN defillama_tvl tvl
ON ch.name = tvl.name
WHERE tvl.date < CURRENT_DATE()
""")

In [8]:
# Function to check out the views

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Don't wrap to multiple lines
pd.set_option('display.max_colwidth', None) # Show full contents of each column

def print_view_sample(conn, view_name, num_rows=5):
    print(f"\n--- {view_name.upper()} ---\n")
    
    # Fetch 5 random rows and convert to pandas DataFrame
    query = f"SELECT * FROM {view_name} ORDER BY RANDOM() LIMIT {num_rows}"
    df = conn.query(query).to_df()
    
    # Print column names
    print("Columns:", ", ".join(df.columns))
    
    # Print the DataFrame
    print("\nRandom Sample Rows:")
    print(df)

def print_view_to_csv(conn, view_name, output_file):
    output_file_csv = f"{output_file}.csv"
    print(f"\nExporting sample from {view_name.upper()} to {output_file_csv}")
    
    # Fetch 5 random rows and convert to pandas DataFrame
    query = f"SELECT * FROM {view_name}"
    df = conn.query(query).to_df()
    
    # Export DataFrame to CSV
    df.to_csv(f"processed/{output_file_csv}", index=False)
    print(f"Sample exported to {output_file_csv}")

In [9]:
# List of views
views = ['growthepie', 'l2beat', 'defillama']

# Print sample for each view
for view in views:
    print_view_sample(conn, view)


--- GROWTHEPIE ---

Columns: date, index, origin_key, costs_blobs_eth, costs_blobs_usd, costs_l1_eth, costs_l1_usd, costs_total_eth, costs_total_usd, daa, fdv_eth, fdv_usd, fees_paid_eth, fees_paid_usd, gas_per_second, market_cap_eth, market_cap_usd, profit_eth, profit_usd, rent_paid_eth, rent_paid_usd, stables_mcap, stables_mcap_eth, tvl, tvl_eth, txcosts_median_eth, txcosts_median_usd, txcount, name, url_key, chain_type, caip2, evm_chain_id, deployment, chain_name, description, da_layer, symbol, bucket, ecosystem, colors, logo, technology, purpose, launch_date, enable_contracts, l2beat_stage, l2beat_link, l2beat_id, raas, stack, website, twitter, block_explorer, block_explorers, rhino_listed, rhino_naming, stack_label, gtp_min_date

Random Sample Rows:
        date  index  origin_key  costs_blobs_eth  costs_blobs_usd  \
0 2024-06-02   6301         imx              NaN              NaN   
1 2024-03-25  11190        mode     1.677722e-11     5.795282e-08   
2 2024-08-26  14097    reds

In [10]:
# Create unique metadata
chain_metadata_df = conn.query("""
WITH get_metadata_list AS (
    SELECT
        -- unify naming and chain id
        COALESCE(l2b.name, gtp.name, dfl.name) AS name_mapped
        , COALESCE(/*l2b.chain_id, */gtp.evm_chain_id, dfl.chainId) AS chain_id_mapped

        ---
        -- metadata - do as aggregations to avoid duplicates
        ---

        , MAX(
            COALESCE(
            gtp.chain_type,
            UPPER(REPLACE(l2b.type,'ayer',''))
            )
            ) AS layer
        , MAX(l2b.stage) AS stage
        , MAX(l2b.purposes) AS purposes
        , MAX(COALESCE(l2b.raas, gtp.raas)) AS raas
        , MAX(l2b.stack_name) AS stack_name
        , MAX(l2b.da_layer) AS da_layer
        , MAX(l2b.infra) AS infra_label
        
        , MAX(CASE WHEN dfl.name IS NOT NULL THEN true ELSE false END) AS is_defillama
        , MAX(CASE WHEN gtp.chain_name IS NOT NULL THEN true ELSE false END) AS is_growthepie
        , MAX(CASE WHEN l2b.name IS NOT NULL THEN true ELSE false END) AS is_l2beat

    FROM defillama_chains dfl
    FULL OUTER JOIN growthepie_metadata gtp
        ON replace(lower(dfl.name),' ','') = replace(lower(gtp.chain_name),' ','')
        OR dfl.chainId = gtp.evm_chain_id
    FULL OUTER JOIN l2beat_summary l2b
        ON replace(lower(dfl.name),' ','') = replace(lower(l2b.name),' ','')
        OR replace(lower(gtp.name),' ','') = replace(lower(l2b.name),' ','')
        -- l2beat does not provide chain id
    group by 1,2
    )

    SELECT *
    -- if L2Beat or Growthepie has the chain, assume it's Ethereum Ecosystem
    , CASE 
            WHEN is_growthepie OR is_l2beat
                OR (name_mapped = 'Ethereum') 
            THEN true 
            ELSE false 
        END AS is_ethereum_ecosystem
    FROM get_metadata_list
""").to_df()

In [11]:
# See sample chain metadata
chain_metadata_df[chain_metadata_df['layer']=='L2'].sample(5)

,name_mapped,chain_id_mapped,layer,stage,purposes,raas,stack_name,da_layer,infra_label,is_defillama,is_growthepie,is_l2beat,is_ethereum_ecosystem
198,Boba Network,NaN,L2,Stage 0,['Universal'],None,OPStack,EthereumBlobs,None,False,False,True,True
384,ZKSpace,NaN,L2,Stage 0,"['NFT', 'Exchange', 'Payments']",None,None,EthereumCalldata,None,False,False,True,True
341,Re.al,111188.0,L2,NotApplicable,"['Universal', 'RWA']",Gelato,Orbit,DAC,None,True,False,True,True
105,Nal,NaN,L2,UnderReview,['Universal'],None,None,None,None,False,False,True,True
101,Term Structure,NaN,L2,Stage 0,"['Payments', 'Exchange', 'Lending']",None,None,EthereumCalldata,None,False,False,True,True


In [12]:
# Generate date range
if get_start_of_month:
    start_date = (datetime.now().replace(day=1) - timedelta(days=trailing_days)).date()
else:
    start_date = (datetime.now() - timedelta(days=trailing_days)).date()

end_date = datetime.now().date()
date_range = pd.date_range(start=start_date, end=end_date).date

# Convert date_range to DataFrame
date_range_df = pd.DataFrame(date_range, columns=['date'])

# Repeat chain metadata for each date
chain_metadata_expanded = chain_metadata_df.assign(key=1).merge(date_range_df.assign(key=1), on='key').drop('key', axis=1)

# Load in to a duckdb table
conn.register('chain_metadata_expanded', chain_metadata_expanded)

In [13]:
print(len(chain_metadata_expanded))

145080


In [14]:
# Build Dataset

all_data_df = conn.query("""
SELECT
  md.date
, name_mapped

-- Chain Activity Metrics
, COALESCE(l2b.transactions_per_day, gtp.txcount) AS transactions_per_day
, gtp.costs_blobs_eth, gtp.costs_blobs_usd, gtp.costs_l1_eth, gtp.costs_l1_usd, gtp.costs_total_eth, gtp.costs_total_usd
, gtp.fees_paid_eth, gtp.fees_paid_usd
, gtp.profit_eth, gtp.profit_usd
, gtp.gas_per_second, gtp.txcosts_median_eth, gtp.txcosts_median_usd

-- Onchain Value Metrics
, dfl.tvl AS app_tvl_usd
, dfl.tvl/l2b.ethPrice AS app_tvl_eth

, l2b.native AS native_assets_onchain_usd, l2b.native/l2b.ethPrice AS native_assets_onchain_eth
, l2b.canonical AS canonical_assets_onchain_usd, l2b.canonical/l2b.ethPrice AS canonical_assets_onchain_eth
, l2b.external AS external_assets_onchain_usd, l2b.external/l2b.ethPrice AS external_assets_onchain_eth

, l2b.native+l2b.canonical+l2b.external AS total_assets_onchain_usd
, (l2b.native+l2b.canonical+l2b.external)/l2b.ethPrice AS total_assets_onchain_eth

-- Metadata
, md.chain_id_mapped, md.layer, md.stage, md.purposes, md.raas, md.stack_name, md.da_layer, md.infra_label
, md.is_defillama, md.is_growthepie, md.is_l2beat, md.is_ethereum_ecosystem


FROM chain_metadata_expanded md
LEFT JOIN growthepie gtp
    ON (
        replace(lower(md.name_mapped),' ','') = replace(lower(gtp.chain_name),' ','')
        OR (md.chain_id_mapped = gtp.evm_chain_id AND md.chain_id_mapped IS NOT NULL
            )
        )
    AND md.date = gtp.date
    AND md.date >= gtp.gtp_min_date

LEFT JOIN l2beat l2b
    ON (
        replace(lower(md.name_mapped),' ','') = replace(lower(l2b.name),' ','')
        -- OR md.chain_id_mapped = l2b.chain_id  -- l2beat does not provide chainid
        )
    AND md.date = l2b.date
    AND md.date >= l2b.l2b_min_date

LEFT JOIN defillama dfl
    ON (
        replace(lower(md.name_mapped),' ','') = replace(lower(dfl.name),' ','')
        OR (md.chain_id_mapped = dfl.chainId AND md.chain_id_mapped IS NOT NULL
            )
        )
    AND md.date = dfl.date
    AND md.date >= dfl.dfl_min_date

-- Remove days with no data
WHERE COALESCE(gtp.date,l2b.date,dfl.date) IS NOT NULL

""").to_df()

BinderException: Binder Error: Values list "gtp" does not have a column named "l2b_min_date"

In [ ]:
all_data_df[(all_data_df['name_mapped'] == 'Arbitrum One') & (all_data_df['date'] == '2024-11-05')].head(5)